In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [6]:
import pandas as pd

# Use the 'error_bad_lines=False' argument to skip problematic lines
df_fake = pd.read_csv("/content/Fake.csv", on_bad_lines='skip',engine='python')
df_true = pd.read_csv("/content/True.csv", on_bad_lines='skip',engine='python')


In [7]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
df_true.head(5)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
df_fake["class"] = 0
df_true["class"] = 1

In [10]:
df_fake.shape, df_true.shape

((9184, 5), (6860, 5))

In [12]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
# Get the last 10 index labels
last_10_indices = df_fake.index[-10:]
# Drop rows by index labels
df_fake.drop(last_10_indices, axis=0, inplace=True)


df_true_manual_testing = df_true.tail(10)
# Get the last 10 index labels
last_10_indices = df_true.index[-10:]
# Drop rows by index labels
df_true.drop(last_10_indices, axis=0, inplace=True)

In [13]:
df_fake.shape, df_true.shape

((9174, 5), (6850, 5))

In [14]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-14-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-14-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [15]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
9174,ERIC HOLDER DOUBLES DOWN with Second Threat Ag...,Eric Holder just doubled down on threats this ...,politics,"Dec 17, 2017",0
9175,BERNIE SANDERS Just Proved His Cluelessness on...,"Sen. Bernie Sanders (I., Vt.) said Sunday Demo...",politics,"Dec 17, 2017",0
9176,WATCH: Democrat Heads Are Exploding After What...,The newly elected US Senator Doug Jones (D-AL...,politics,"Dec 17, 2017",0
9177,JUST IN: DISGRACED DEMOCRAT HARRY REID Funnele...,The Defense Department secretly set up a progr...,politics,"Dec 17, 2017",0
9178,IT JUST GOT REAL! GOP Rep. Jim Jordan Tells Ju...,One of the big players in trying to get to the...,politics,"Dec 17, 2017",0
9179,BIZARRE 2006 FILM Starring Former White House ...,There are conflicting reports about why the ou...,politics,"Dec 16, 2017",0
9180,FOX NEWS BOMBSHELL REPORT: Trump Transition Te...,"On Friday, Democrats were doing their best to ...",politics,"Dec 16, 2017",0
9181,CHAFFETZ CALLS FOR SESSIONS TO RESIGN: “It’s t...,Jason Chaffetz just went there! He said what e...,politics,"Dec 16, 2017",0
9182,JUST IN: Mueller Improperly Obtained Transitio...,This bombshell just in is more evidence of the...,politics,"Dec 16, 2017",0
9183,DIAMOND & SILK: Liberals Respect President Tru...,Diamond and Silk: Liberals are trying to dest...,politics,"Dec 16, 2017",0


In [16]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
6850,China's CIC head says Trump to be careful in c...,MELBOURNE (Reuters) - The CEO of China’s sover...,politicsNews,"December 8, 2016",1
6851,U.S. Republicans quibble over effective date o...,WASHINGTON (Reuters) - U.S. Republican senator...,politicsNews,"December 7, 2016",1
6852,Trump to pick foe of Obama climate agenda to r...,WASHINGTON (Reuters) - Donald Trump will name ...,politicsNews,"December 7, 2016",1
6853,U.S. mayors ask Trump to keep young illegal im...,NEW YORK (Reuters) - Mayors from the largest U...,politicsNews,"December 7, 2016",1
6854,Bank lobby presses for commission to lead cons...,WASHINGTON (Reuters) - Bank lobbying groups ar...,politicsNews,"December 7, 2016",1
6855,Trump picks longtime friend of Beijing as U.S....,BEIJING/NEW YORK (Reuters) - President-elect D...,politicsNews,"December 7, 2016",1
6856,Trump chooses pro wrestling magnate Linda McMa...,WASHINGTON (Reuters) - U.S. President-elect Do...,politicsNews,"December 7, 2016",1
6857,Trump chooses pro wrestling magnate Linda McMa...,WASHINGTON (Reuters) - U.S. President-elect Do...,politicsNews,"December 7, 2016",1
6858,Kelly confirms he's been asked to be U.S. home...,(Reuters) - Retired Marine Corps General John ...,politicsNews,"December 7, 2016",1
6859,Trump says 'nothing funny' about 'Saturday Nig...,NEW YORK (Reuters) - U.S. President-elect Dona...,politicsNews,"December 7, 2016",1


In [17]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [18]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [19]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [20]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [21]:
df.isnull().sum()

,0
text,0
class,0


In [22]:
df = df.sample(frac = 1)

In [23]:
df.head()

,text,class
6768,(Reuters) - No Wall Street firm is set to do b...,1
4497,"Sarah Palin, one of the nigh-innumerable depl...",0
19,(Reuters) - A gift-wrapped package addressed t...,1
8283,Marco Rubio was looming large in New Hampshire...,0
799,"Donald Trump s White House is in chaos, and th...",0


In [24]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [26]:
df.columns

Index(['text', 'class'], dtype='object')

In [27]:
df.head()

,text,class
0,(Reuters) - No Wall Street firm is set to do b...,1
1,"Sarah Palin, one of the nigh-innumerable depl...",0
2,(Reuters) - A gift-wrapped package addressed t...,1
3,Marco Rubio was looming large in New Hampshire...,0
4,"Donald Trump s White House is in chaos, and th...",0


In [28]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [29]:
df["text"] = df["text"].apply(wordopt)

In [30]:
x = df["text"]
y = df["class"]

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [33]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [34]:
pred_lr=LR.predict(xv_test)

In [35]:
LR.score(xv_test, y_test)

0.9930104842735896

In [36]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2291
           1       0.99      0.99      0.99      1715

    accuracy                           0.99      4006
   macro avg       0.99      0.99      0.99      4006
weighted avg       0.99      0.99      0.99      4006



In [37]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [38]:
pred_dt = DT.predict(xv_test)

In [39]:
DT.score(xv_test, y_test)

0.9985022466300549

In [40]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2291
           1       1.00      1.00      1.00      1715

    accuracy                           1.00      4006
   macro avg       1.00      1.00      1.00      4006
weighted avg       1.00      1.00      1.00      4006



In [47]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [48]:
pred_gbc = GBC.predict(xv_test)

In [49]:
GBC.score(xv_test, y_test)

0.99900149775337

In [50]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2291
           1       1.00      1.00      1.00      1715

    accuracy                           1.00      4006
   macro avg       1.00      1.00      1.00      4006
weighted avg       1.00      1.00      1.00      4006



In [51]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [52]:
pred_rfc = RFC.predict(xv_test)

In [53]:
RFC.score(xv_test, y_test)

0.9947578632051922

In [54]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2291
           1       1.00      0.99      0.99      1715

    accuracy                           0.99      4006
   macro avg       0.99      0.99      0.99      4006
weighted avg       0.99      0.99      0.99      4006



In [55]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [56]:
news = str(input())
manual_testing(news)

BRUSSELS (Reuters) - NATO allies on Tuesday welcomed President Donald Trump s decision to commit more forces to Afghanistan, as part of a new U.S. strategy he said would require more troops and funding from America s partners. Having run for the White House last year on a pledge to withdraw swiftly from Afghanistan, Trump reversed course on Monday and promised a stepped-up military campaign against  Taliban insurgents, saying:  Our troops will fight to win .  U.S. officials said he had signed off on plans to send about 4,000 more U.S. troops to add to the roughly 8,400 now deployed in Afghanistan. But his speech did not define benchmarks for successfully ending the war that began with the U.S.-led invasion of Afghanistan in 2001, and which he acknowledged had required an   extraordinary sacrifice of blood and treasure .  We will ask our NATO allies and global partners to support our new strategy, with additional troops and funding increases in line with our own. We are confident they w

In [57]:
news = str(input())
manual_testing(news)

e editor, or simply post comments online, to try to fool you into thinking an independent or grassroots movement is speaking. ~ Sharyl Attkisson, Investigative ReporterHow do you separate fact from fiction? Sharyl Attkisson finishes her talk with some insights on how to identify signs of propaganda and astroturfing  These methods are used to give people the impression that there is widespread support for an agenda, when, in reality, one may not exist. Astroturf tactics are also used to discredit or criticize those that disagree with certain agendas, using stereotypical names such as conspiracy theorist or quack. When in fact when someone dares to reveal the truth or questions the  official  story, it should spark a deeper curiosity and encourage further scrutiny of the information.This article (Journalist Reveals Tactics Brainwashing Industry Uses to Manipulate the Public) was originally created and published by Waking Times and is published here under a Creative Commons license with a